# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
print(ed.links, ed.get_contents())

['https://edwarddonner.com/', 'https://edwarddonner.com/connect-four/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonne

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/Qwen/QwQ-32B',
 '/google/gemma-3-27b-it',
 '/deepseek-ai/DeepSeek-R1',
 '/SparkAudio/Spark-TTS-0.5B',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/ASLP-lab/DiffRhythm',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/facebook/natural_reasoning',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/SmallDoge/SmallThoughts',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
#print(get_all_details("https://huggingface.co"))
print(get_all_details("https://www.capitalone.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.capitalone.com/about/'}, {'type': 'corporate information page', 'url': 'https://www.capitalone.com/about/corporate-information/'}, {'type': 'newsroom page', 'url': 'https://www.capitalone.com/about/newsroom/'}, {'type': 'careers page', 'url': 'https://www.capitalonecareers.com/'}, {'type': 'diversity page', 'url': 'https://www.capitalone.com/diversity/'}, {'type': 'help center page', 'url': 'https://www.capitalone.com/help-center/'}, {'type': 'small business homepage', 'url': 'https://www.capitalone.com/small-business/homepage/'}, {'type': 'commercial page', 'url': 'https://www.capitalone.com/commercial/'}, {'type': 'banking page', 'url': 'https://www.capitalone.com/bank/'}]}
Landing page:
Webpage Title:
Capital One | Credit Cards, Checking, Savings & Auto Loans
Webpage Contents:
Skip to main content
Sign In
Credit Cards
Checking & Savings
Auto
Business
Commercial
Benefits & Tools
Search
Help Center
Locations
Sign In
Cr

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
#get_brochure_user_prompt("HuggingFace", "https://huggingface.co")
get_brochure_user_prompt("Capital One", "https://www.capitalone.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.capitalone.com/about/'}, {'type': 'careers page', 'url': 'https://www.capitalonecareers.com/'}, {'type': 'company information', 'url': 'https://www.capitalone.com/about/corporate-information/'}, {'type': 'press page', 'url': 'https://www.capitalone.com/about/newsroom/'}, {'type': 'diversity page', 'url': 'https://www.capitalone.com/diversity/'}]}


"You are looking at a company called: Capital One\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nCapital One | Credit Cards, Checking, Savings & Auto Loans\nWebpage Contents:\nSkip to main content\nSign In\nCredit Cards\nChecking & Savings\nAuto\nBusiness\nCommercial\nBenefits & Tools\nSearch\nHelp Center\nLocations\nSign In\nCredit Cards\nTypes of Credit Cards\nCredit Building\nCredit Level: Fair to Rebuilding\nCash Back Rewards\nCredit Level: Excellent to Fair\nTravel Rewards\nCredit Level: Excellent to Good\nStudent Rewards\nCredit Level: Fair\nBusiness Rewards\nCredit Level: Excellent to Fair\nCredit card overview\nCredit card overview\nFind a Credit Card\nSee if You're Pre-Approved\nCompare All Cards\nRedeem a Mail Offer\nTips & Tools\nCredit Card Benefits\nCapital One Travel\nCapital One Shopping\nTrack Credit with CreditWise®\nCredit Card FAQs\nCommon 

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
#create_brochure("HuggingFace", "https://huggingface.co")
create_brochure("Capital One", "https://www.capitalone.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.capitalone.com/about/'}, {'type': 'company information', 'url': 'https://www.capitalone.com/about/corporate-information/'}, {'type': 'newsroom', 'url': 'https://www.capitalone.com/about/newsroom/'}, {'type': 'careers page', 'url': 'https://www.capitalonecareers.com/'}, {'type': 'diversity page', 'url': 'https://www.capitalone.com/diversity/'}, {'type': 'financial products', 'url': 'https://www.capitalone.com/bank/'}, {'type': 'credit card offerings', 'url': 'https://www.capitalone.com/credit-cards/'}, {'type': 'auto financing page', 'url': 'https://www.capitalone.com/cars/auto-financing'}, {'type': 'small business page', 'url': 'https://www.capitalone.com/small-business/homepage/'}, {'type': 'commercial services', 'url': 'https://www.capitalone.com/commercial/'}]}


# Welcome to Capital One

## Overview
At Capital One, we provide a wide range of financial products, including credit cards, checking and savings accounts, auto loans, and commercial banking solutions. Our mission is to redefine the banking experience with innovative technology and outstanding customer service.

---

## Products and Services

### **Credit Cards**
- **Types Available:**
  - Cash Back Rewards
  - Travel Rewards
  - Student Rewards
  - Business Rewards
- **Key Features:**
  - Customized offerings based on credit level
  - Easy online application and management tools, including pre-approval options and redemption opportunities.

### **Checking & Savings**
- **Savings Accounts:**
  - 360 Performance Savings™ with competitive rates
  - 360 CDs® for fixed-rate terms
  - Kids Savings Accounts for parental guidance
- **Checking Accounts:**
  - 360 Checking® with no fees and access 24/7
  - MONEY Teen Checking for educación y ahorro

### **Auto Loans**
- **Auto Navigator:** 
  - Shop for cars nationwide and pre-qualify without affecting your credit score.
  - Options for auto refinance to save on payments.

### **Business Solutions**
- **Business Banking:**
  - Various business credit cards including cash back and travel rewards.
  - Business checking and savings to assist at any stage of your enterprise.

### **Commercial Banking**
- Tailored solutions for various industries, including community finance and treasury management.

---

## Company Culture
At Capital One, we believe in a culture of collaboration, innovation, and inclusion. Our team thrives in an environment that values diverse opinions and encourages different perspectives. We promote a supportive atmosphere that emphasizes growth, creativity, and continuous learning. 

### **Community Engagement**
- Considerable focus on community impact through initiatives like the Capital One Impact Initiative, which promotes financial education and support.

---

## Careers at Capital One
Capital One offers a dynamic work environment with opportunities for career growth in various fields. We seek innovative thinkers who reflect our commitment to technology and customer service.

### **Why Join Us?**
- **Competitive Salaries & Benefits:** Comprehensive packages designed to meet the needs of our employees.
- **Learning & Development:** Ongoing opportunities in professional development through training initiatives and workshops.
- **Work-Life Balance:** Flexible work arrangements designed to support your lifestyle.

---

## Our Customers
Serving millions of individuals and businesses across the nation, our diverse customer base includes:
- Everyday consumers seeking easy banking solutions.
- Students beginning their financial journeys.
- Businesses looking for comprehensive banking services to grow and thrive.

---

Explore **Capital One** today and see how we can help you manage your finances with confidence and ease! Visit our website for more information on our products, services, and career opportunities.

**[Meet Your Financial Goals with Capital One](https://www.capitalone.com)**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## About Us
**Hugging Face** is the AI community building the future by providing an innovative platform where the machine learning community collaborates on models, datasets, and applications. With over 1 million models and 250,000 datasets available, we empower professionals and enthusiasts alike to lead the way in machine learning development.

## Our Mission
We strive to enhance collaboration and creativity within the AI ecosystem. Our open-source tools and resources are designed to facilitate access to state-of-the-art machine learning technology for everyone.

## Target Customers
Hugging Face serves a diverse clientele, including:
- **Enterprise Organizations**: Companies like Amazon, Google, Intel, and Microsoft extensively use our platform to host models and deploy AI solutions.
- **Research Institutions**: We provide access to cutting-edge tools and datasets that support advanced research in AI and ML.
- **Individual Developers**: Enthusiasts and aspiring data scientists benefit from our open-source resources to build their portfolios and skills.

## Our Offerings
1. **Models**: Access to over 1M+ models for various applications including natural language processing, image recognition, and more.
2. **Datasets**: Extensive library of over 250,000 datasets available for training and evaluation of models.
3. **Spaces**: Interactive environment for running AI applications and visualizing projects.
4. **Enterprise Solutions**: Robust platform with enterprise-grade security and dedicated support, starting at $20/user/month.

## Company Culture
Hugging Face emphasizes a **collaborative and inclusive culture** where innovation is fostered. We pride ourselves on our open-source mindset, encouraging contributions from the community. Our workspace is designed to facilitate teamwork, creativity, and ongoing learning, ensuring that each team member feels valued and inspired.

## Career Opportunities
We are always on the lookout for passionate individuals to join our team! Our careers page features multiple openings across different fields. Be a part of a vibrant community that is pioneering the future of AI. Explore current openings and apply to join our mission today.

## Join Us
Become a part of the Hugging Face community! Whether you are looking to leverage AI in your organization, contribute to groundbreaking research, or build your own projects with the latest technology, Hugging Face offers something for everyone.

For more information, visit us at [Hugging Face](https://huggingface.co).

--- 

*Follow us on our social channels: [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/hugging-face), [Discord](https://discord.com/invite/huggingface).*

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face – The AI Community Building the Future

At Hugging Face, we are dedicated to fostering a global community of machine learning enthusiasts and professionals. Our platform serves as a collaborative space for developers and researchers to share models, datasets, and applications that push the boundaries of artificial intelligence.

---

## What We Do

- **Collaborative Platform**: Create, discover, and collaborate on over 1 million models and more than 250,000 datasets tailored for machine learning (ML) activities.
- **AI Tools**: Utilize state-of-the-art tools including Transformers and Diffusers to excel in various modalities such as text, image, video, audio, and 3D.
- **Enterprise Solutions**: We offer advanced deployment options with enterprise-grade security and support tailored for organizations, starting at just $20/user/month.

---

## Our Customers

Join more than **50,000 organizations** including key players such as:
- Google
- Amazon
- Microsoft
- Facebook (Meta)
- Grammarly

These entities leverage our innovative technology to transform their AI projects and solutions efficiently.

---

## Company Culture

At Hugging Face, we believe in building a nurturing and inclusive culture where creativity thrives. Our community-centric approach fosters collaboration, innovation, and transparency. We thrive on shared knowledge, diverse backgrounds, and collective growth.

---

## Careers at Hugging Face

We are always looking for talented individuals to join us in our mission to make machine learning accessible and collaborative for everyone. **Check out our open positions** to find out how you can contribute to our vision and be a part of our growing team!

### Current Openings
Explore various roles across different aspects of our business and become a key player in building the future of AI. 

---

## Get Involved

Are you ready to take part in the future of AI? Whether you are a developer, researcher, or simply an AI enthusiast, you can contribute to our community by:
- **Collaborating on projects or models**
- **Sharing datasets & applications**
- **Exploring AI technologies through our resources**

**Sign Up** today to join our vibrant community and start innovating with Hugging Face!

---

### Connect with Us
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

---

*Join Hugging Face, where we’re building a brighter AI future, together!*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>